In [1]:
import geopandas as gpd
import mercantile
from shapely.geometry import box, Polygon, MultiPolygon
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
import json

In [2]:

def get_bbox_from_polygon(polygon):
    """Return bounding box (minx, miny, maxx, maxy) of a polygon."""
    return polygon.bounds


def get_tiles_from_bbox(bbox, zoom=14):
    """Return mercantile tile list for a bbox at a given zoom."""
    minx, miny, maxx, maxy = bbox
    return list(mercantile.tiles(minx, miny, maxx, maxy, zooms=[zoom]))


def filter_tiles_by_polygon(tiles, polygon):
    """Return only those tiles whose centroid lies inside the polygon."""
    filtered = []
    for tile in tqdm(tiles, desc="Filtering tiles"):
        bounds = mercantile.bounds(tile)
        tile_box = box(bounds.west, bounds.south, bounds.east, bounds.north)
        centroid = tile_box.centroid
        if polygon.intersects(centroid):
            filtered.append(tile)
    return filtered


def plot_tiles_and_boundary(filtered_tiles, boundary_polygon, bundesland_name="Region", save_path=None, output_folder="output",show=False):
    """Plot the filtered tiles and boundary together using projected CRS."""
    # Erstelle GeoDataFrame aus Tiles
    tile_polygons = []
    for tile in filtered_tiles:
        bounds = mercantile.bounds(tile)
        tile_polygons.append(Polygon([
            (bounds.west, bounds.south),
            (bounds.west, bounds.north),
            (bounds.east, bounds.north),
            (bounds.east, bounds.south)
        ]))

    tile_gdf = gpd.GeoDataFrame(geometry=tile_polygons, crs="EPSG:4326")
    boundary_gdf = gpd.GeoDataFrame(geometry=[boundary_polygon], crs="EPSG:4326")

    # Projiziere beides z. B. nach EPSG:3857
    tile_gdf_proj = tile_gdf.to_crs("EPSG:3857")
    boundary_gdf_proj = boundary_gdf.to_crs("EPSG:3857")

    # Plot
    fig, ax = plt.subplots(figsize=(8, 8))
    tile_gdf_proj.plot(ax=ax, color='green', edgecolor='black', alpha=0.5)
    boundary_gdf_proj.plot(ax=ax, color='blue', alpha=0.3, edgecolor='black')

    ax.set_axis_off()
    ax.set_title(f"{bundesland_name} – Tile Coverage ({len(filtered_tiles)} tiles)")

    if save_path is not None:
        os.makedirs(output_folder, exist_ok=True)
        full_path = os.path.join(output_folder, save_path)
        plt.savefig(full_path, bbox_inches='tight', pad_inches=0.1)
        print(f"✔️ Plot saved to: {full_path}")
    
    if show:
        plt.show()
    
    plt.close(fig)  # 🧼 Wichtig: Figure schließen, um Speicher zu sparen





In [3]:
# # Lade Bundesländer
# bland = gpd.read_file("https://raw.githubusercontent.com/isellsoap/deutschlandGeoJSON/main/2_bundeslaender/1_sehr_hoch.geo.json")

# # Wähle Berlin aus
# berlin = bland[bland["id"] == "DE-BE"].dissolve().geometry.iloc[0]

# # Berechne Tiles
# bbox = get_bbox_from_polygon(berlin)
# tiles = get_tiles_from_bbox(bbox, zoom=14)
# filtered_tiles = filter_tiles_by_polygon(tiles, berlin)

# # Plot
# plot_tiles_and_boundary(filtered_tiles, 
#                         berlin, 
#                         bundesland_name="Berlin",
#                         save_path="DE-BE_tiles.png")


In [ ]:
# Iterate through all Bundesländer


def save_tiles_to_json(tiles, bundesland_id, output_folder="output/tile_cache"):
    os.makedirs(output_folder, exist_ok=True)
    tile_list = [{"x": t.x, "y": t.y, "z": t.z} for t in tiles]
    path = os.path.join(output_folder, f"{bundesland_id}_tiles.json")
    with open(path, "w") as f:
        json.dump(tile_list, f)
    print(f"✔️ Tiles saved to: {path}")


# Lade Bundesländer
bland = gpd.read_file("https://raw.githubusercontent.com/isellsoap/deutschlandGeoJSON/main/2_bundeslaender/1_sehr_hoch.geo.json")


for _, row in bland.iterrows():
    name = row["name"]
    b_id = row["id"]  # <- z. B. 'DE-BE'
    polygon = row["geometry"]

    bbox = get_bbox_from_polygon(polygon)
    tiles = get_tiles_from_bbox(bbox, zoom=14)
    filtered_tiles = filter_tiles_by_polygon(tiles, polygon)

    print(f"{b_id} – {name}: {len(filtered_tiles)} tiles")

    save_tiles_to_json(filtered_tiles, b_id)
    
    plot_tiles_and_boundary(
        filtered_tiles,
        polygon,
        bundesland_name=name,
        save_path=f"{b_id}_tiles.png",
        output_folder="output/tile_cache",          # Folder goes here
        show=False
    )


Filtering tiles: 100%|██████████| 21528/21528 [00:00<00:00, 34272.15it/s]


DE-BW – Baden-Württemberg: 13618 tiles
✔️ Tiles saved to: output/tile_cache/DE-BW_tiles.json
✔️ Plot saved to: output/tile_cache/DE-BW_tiles.png


Filtering tiles: 100%|██████████| 50609/50609 [00:01<00:00, 37364.96it/s]


DE-BY – Bayern: 27315 tiles
✔️ Tiles saved to: output/tile_cache/DE-BY_tiles.json
✔️ Plot saved to: output/tile_cache/DE-BY_tiles.png


Filtering tiles: 100%|██████████| 832/832 [00:00<00:00, 49911.48it/s]


...

Filtering tiles: 100%|██████████| 18755/18755 [00:00<00:00, 51458.38it/s]


DE-ST – Sachsen-Anhalt: 9075 tiles
✔️ Tiles saved to: output/tile_cache/DE-ST_tiles.json
✔️ Plot saved to: output/tile_cache/DE-ST_tiles.png


Filtering tiles: 100%|██████████| 15805/15805 [00:00<00:00, 45151.00it/s]


DE-SN – Sachsen: 7778 tiles
✔️ Tiles saved to: output/tile_cache/DE-SN_tiles.json
✔️ Plot saved to: output/tile_cache/DE-SN_tiles.png


Filtering tiles:  80%|████████  | 16875/21014 [00:01<00:00, 14791.04it/s]